# Feature Engineering

In [1]:
import pandas as pd
import numpy as np
import holidays
from sklearn.cross_decomposition import PLSRegression

In [2]:
# --- Load training data (Years 1 and 2) ---
train_df = pd.read_excel('.././datasets/training.xlsx', sheet_name='Data')

# --- Load testing data (Year 3)
test_df = pd.read_excel('.././datasets/testing.xlsx', sheet_name='Data')

## Dimensional Reduction

In [3]:
# --- Step 0: Prepare the Data ---
# For consistency, we'll combine the training and testing sets to apply features.
# We'll fit the PLS model ONLY on the training data to prevent data leakage.
train_rows = len(train_df)
full_df = pd.concat([train_df, test_df], ignore_index=True)

# --- Step 1: PLS Dimensionality Reduction ---
print("Step 1: Performing PLS Dimensionality Reduction...")

# Define predictor sets and the target variable ('Load')
temp_cols = sorted([col for col in full_df.columns if 'Temp' in col])
ghi_cols = sorted([col for col in full_df.columns if 'GHI' in col])
target_col = 'Load'

# Isolate the training data to fit the models
X_train_temp = train_df[temp_cols]
X_train_ghi = train_df[ghi_cols]
y_train = train_df[target_col]

# a) PLS for Temperature
pls_temp = PLSRegression(n_components=1)
pls_temp.fit(X_train_temp, y_train)
full_df['Combined_Temp'] = pls_temp.transform(full_df[temp_cols])

# b) PLS for GHI
pls_ghi = PLSRegression(n_components=1)
pls_ghi.fit(X_train_ghi, y_train)
full_df['Combined_GHI'] = pls_ghi.transform(full_df[ghi_cols])

print("PLS transformation complete. 'Combined_Temp' and 'Combined_GHI' created.")

Step 1: Performing PLS Dimensionality Reduction...
PLS transformation complete. 'Combined_Temp' and 'Combined_GHI' created.


## New Feature Generation

### Time Features

In [4]:
# Create mappings for month-day calculations
days_in_month = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
cumulative_days_map = {i: sum(list(days_in_month.values())[:i-1]) for i in range(1, 13)}

# Clean data and convert to integer
full_df.dropna(subset=['Month', 'Day'], inplace=True)
full_df['Month'] = full_df['Month'].astype(int)
full_df['Day'] = full_df['Day'].astype(int)

# Calculate DayOfYear and DayOfWeek
max_days = full_df['Month'].map(days_in_month)
full_df['Day_Clipped'] = np.minimum(full_df['Day'], max_days)
cumulative_days = full_df['Month'].map(cumulative_days_map)
full_df['DayOfYear'] = cumulative_days + full_df['Day_Clipped']
full_df['DayOfWeek'] = (full_df['DayOfYear'] - 1) % 7
# --- End of Fix ---


# 1. Sinusoidal Encodings
P_day = 24
P_week = 168
P_year = 8766 # 365.25 * 24

# Create a continuous time index 't'
full_df['Timestep'] = np.arange(len(full_df))
t = full_df['Timestep']

# Daily patterns (k=2)
full_df['sin_day'] = np.sin(2 * np.pi * 2 * t / P_day)
full_df['cos_day'] = np.cos(2 * np.pi * 2 * t / P_day)

# Weekly patterns (k=2)
full_df['sin_week'] = np.sin(2 * np.pi * 2 * t / P_week)
full_df['cos_week'] = np.cos(2 * np.pi * 2 * t / P_week)

# Yearly/Seasonal patterns (k=4)
full_df['sin_year'] = np.sin(2 * np.pi * 4 * t / P_year)
full_df['cos_year'] = np.cos(2 * np.pi * 4 * t / P_year)

# 2. Day-of-Week Encoding
# 'DayOfWeek' (0=Mon, 6=Sun) now exists, so we can proceed.
# We'll rename it for clarity.
full_df.rename(columns={'DayOfWeek': 'dow'}, inplace=True)

# Weekday (0) vs. Weekend (1)
full_df['is_weekend'] = full_df['dow'].apply(lambda d: 1 if d >= 5 else 0)

# Holiday encoding (0=No, 1=Yes)
# We map Year 1, 2, 3 to actual years to use the holiday library.
year_map = {1: 2021, 2: 2022, 3: 2023}
full_df['Date'] = pd.to_datetime(full_df['Year'].map(year_map).astype(str) + '-' +
                               full_df['Month'].astype(str) + '-' +
                               full_df['Day'].astype(str), errors='coerce')

us_holidays = holidays.US(state='CA', years=[2021, 2022, 2023])
full_df['is_holiday'] = full_df['Date'].isin(us_holidays).astype(int)

print("Time features created successfully.")

Time features created successfully.


C:\Users\quang\AppData\Local\Temp\ipykernel_23992\3994866085.py:56: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  full_df['is_holiday'] = full_df['Date'].isin(us_holidays).astype(int)


### Weather Features

In [5]:
# 3. Weather Lags and Deltas for Combined_Temp
full_df['lag_1_temp'] = full_df['Combined_Temp'].shift(1)
full_df['lag_24_temp'] = full_df['Combined_Temp'].shift(24)
full_df['delta_1_temp'] = full_df['Combined_Temp'].diff(1)
full_df['delta_24_temp'] = full_df['Combined_Temp'].diff(24)

# Lags and Deltas for Combined_GHI
full_df['lag_1_ghi'] = full_df['Combined_GHI'].shift(1)
full_df['lag_24_ghi'] = full_df['Combined_GHI'].shift(24)
full_df['delta_1_ghi'] = full_df['Combined_GHI'].diff(1)
full_df['delta_24_ghi'] = full_df['Combined_GHI'].diff(24)
print("Weather feature engineering complete.")

Weather feature engineering complete.


### Behavioral Features

In [6]:
# 4. Heating/Cooling Degree Hours
T_base = 20.0
full_df['CDH'] = (full_df['Combined_Temp'] - T_base).clip(lower=0)
full_df['HDH'] = (T_base - full_df['Combined_Temp']).clip(lower=0)

print("Feature creation complete.")

# --- Step 3: Finalize and Save ---
lag_delta_cols = [
    'lag_1_temp', 'lag_24_temp', 'delta_1_temp', 'delta_24_temp',
    'lag_1_ghi', 'lag_24_ghi', 'delta_1_ghi', 'delta_24_ghi'
]
full_df[lag_delta_cols] = full_df[lag_delta_cols].fillna(0)
print("\nNaN values in lag/delta columns have been imputed with 0.")

final_features_df = full_df.copy()

# Save the resulting dataframe to a new file
output_filename = '../datasets/features_engineered.csv'
final_features_df.to_csv(output_filename, index=False)

print(f"Engineered features saved to '{output_filename}'.")
print("\nDataFrame Info:")
final_features_df.info()
print("\nFirst 5 rows of the new features (with NaNs imputed):")
# Display a subset of the new features for review
display(final_features_df[['Date', 'Load', 'Combined_Temp', 'Combined_GHI', 'sin_day', 'cos_day', 'is_weekend', 'is_holiday', 'lag_24_temp', 'delta_1_temp', 'CDH', 'HDH']].head())

# Load the dataset with all engineered features
df = pd.read_csv(output_filename)

# Find all the original site-specific columns to remove
cols_to_drop = [col for col in df.columns if 'Site' in col and ('Temp' in col or 'GHI' in col)]

# Drop the columns
df_final = df.drop(columns=cols_to_drop)

# Save the final, clean dataset
output_filename = '../datasets/features_final.csv'
df_final.to_csv(output_filename, index=False)

print(f"Removed {len(cols_to_drop)} original site-specific columns.")
print(f"Final feature set saved to '{output_filename}'.")

# Display the remaining columns to confirm
print("\nFinal columns in the dataset:")
print(df_final.columns.to_list())

Feature creation complete.

NaN values in lag/delta columns have been imputed with 0.
Engineered features saved to '../datasets/features_engineered.csv'.

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26304 entries, 0 to 26303
Data columns (total 40 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Year           26304 non-null  int64         
 1   Month          26304 non-null  int64         
 2   Day            26304 non-null  int64         
 3   Hour           26304 non-null  int64         
 4   Load           17544 non-null  float64       
 5   Site-1 Temp    26304 non-null  float64       
 6   Site-2 Temp    26304 non-null  float64       
 7   Site-3 Temp    26304 non-null  float64       
 8   Site-4 Temp    26304 non-null  float64       
 9   Site-5 Temp    26304 non-null  float64       
 10  Site-1 GHI     26304 non-null  int64         
 11  Site-2 GHI     26304 non-null  int64         
 12  S

,Date,Load,Combined_Temp,Combined_GHI,sin_day,cos_day,is_weekend,is_holiday,lag_24_temp,delta_1_temp,CDH,HDH
0,2021-01-01,1997.0,-4.042279,-1.646135,0.000000,1.000000e+00,0,1,0.0,0.000000,0.0,24.042279
1,2021-01-01,1921.0,-4.137674,-1.646135,0.500000,8.660254e-01,0,1,0.0,-0.095395,0.0,24.137674
2,2021-01-01,1861.0,-4.205329,-1.646135,0.866025,5.000000e-01,0,1,0.0,-0.067655,0.0,24.205329
3,2021-01-01,1833.0,-4.409676,-1.646135,1.000000,6.123234e-17,0,1,0.0,-0.204347,0.0,24.409676
4,2021-01-01,1847.0,-4.435790,-1.646135,0.866025,-5.000000e-01,0,1,0.0,-0.026113,0.0,24.435790


Removed 10 original site-specific columns.
Final feature set saved to '../datasets/features_final.csv'.

Final columns in the dataset:
['Year', 'Month', 'Day', 'Hour', 'Load', 'Combined_Temp', 'Combined_GHI', 'Day_Clipped', 'DayOfYear', 'dow', 'Timestep', 'sin_day', 'cos_day', 'sin_week', 'cos_week', 'sin_year', 'cos_year', 'is_weekend', 'Date', 'is_holiday', 'lag_1_temp', 'lag_24_temp', 'delta_1_temp', 'delta_24_temp', 'lag_1_ghi', 'lag_24_ghi', 'delta_1_ghi', 'delta_24_ghi', 'CDH', 'HDH']
